# Setup
Setting up connections to AI Studio

In [ ]:
import json

# Import required libraries
from promptflow.azure import PFClient

# Import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from evaluate import run_azure_flow, run_azure_eval_flow

In [ ]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [ ]:
config_path = "../config.json"
pf_azure_client = PFClient.from_config(credential=credential, path=config_path)

# Testing Contoso Chat w/ Sales Data
Running base flow

In [ ]:
# Update the runtime to the name of the runtime you created previously
runtime = "automatic"
flow = "../contoso-chat"
data = "../data/salestestdata.jsonl"
run_name = "contoso-chat-sales"
column_mapping = {"customerId": "${data.customerId}", "question": "${data.question}"}

In [ ]:
base_run = run_azure_flow(
    runtime, flow, run_name, data, column_mapping, pf_azure_client
)
pf_azure_client.stream(base_run)

In [ ]:
details = pf_azure_client.get_details(base_run)
details.head(10)

Running Evaluation

In [ ]:
runtime = "automatic"
eval_flow = "multi_flow/"
data = "../data/salestestdata.jsonl"
run_name = "contoso-chat-sales-eval"

column_mapping = {
    # reference data
    "customerId": "${data.customerId}",
    "question": "${data.question}",
    "context": "${run.outputs.context}",
    # reference the run's output
    "answer": "${run.outputs.answer}",
}

In [ ]:
eval_run = run_azure_eval_flow(
    runtime, eval_flow, run_name, data, column_mapping, base_run, pf_azure_client
)
pf_azure_client.stream(eval_run)

In [ ]:
details = pf_azure_client.get_details(eval_run)
details.head(10)

In [ ]:
metrics = pf_azure_client.get_metrics(eval_run)
print(json.dumps(metrics, indent=4))

In [ ]:
pf_azure_client.visualize([base_run, eval_run])